In [1]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
# os.environ['PYSPARK_SUBMIT_ARGS']="--master –driver-memory 200G pyspark-shell --executor-memory 200G"

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer
from pyspark.ml.feature import PCA
from pyspark.sql import functions as F

In [3]:
# 1. Loading the dataset in PySpark
# spark = SparkSession.builder.appName("RecommendationSystem").getOrCreate()

spark = SparkSession.builder \
    .master('local[*]') \
    .config("spark.driver.memory", "400g") \
    .appName('my-cool-app') \
    .getOrCreate()

# Load the dataset from CSV
data = spark.read.csv("data/data.csv", header=True, inferSchema=True)

23/12/01 21:19:25 WARN Utils: Your hostname, uri-cpu003 resolves to a loopback address: 127.0.1.1; using 10.100.110.3 instead (on interface eno1)
23/12/01 21:19:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/01 21:19:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
nd = data.select(data['productID'],data['rating'],data['reviewerID'])
nd.show()

+----------+------+--------------+
| productID|rating|    reviewerID|
+----------+------+--------------+
|0001713353|   5.0|A1REUF3A1YCPHM|
|0001713353|   5.0| AVP0HXC9FG790|
|0001713353|   5.0|A324TTUBKTN73A|
|0001713353|   5.0|A2RE7WG349NV5D|
|0001713353|   5.0|A32B7QIUDQCD0E|
|0001713353|   5.0|A3H9YD6K9TVKDP|
|0001713353|   5.0| A3M314LI9OYME|
|0001713353|   5.0|A1YDQQJDRHM0FJ|
|0001713353|   5.0| AHXMDCU0N15TN|
|0001713353|   5.0|A2M46WTE5TR5WN|
|0001713353|   5.0| ATHTCOG6BB6WK|
|0001713353|   5.0|A3MA68F62DFNVU|
|0001713353|   3.0|A294QSAEH1Z7YI|
|0001713353|   5.0| AC4I57VYBGO8P|
|0001713353|   5.0|A1RR8Q97PANN98|
|0001713353|   5.0|A1SDAYRVRR62ZH|
|0001061240|   5.0|A1IG9N5URR82EB|
|0001061240|   5.0| A2CVLIZ9ELU88|
|0001061240|   5.0|A2LGACKSC0MALY|
|0001061240|   5.0|  A6EQG0P75KHJ|
+----------+------+--------------+
only showing top 20 rows



In [5]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [6]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col
indexer = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in list(set(nd.columns)-set(['rating'])) ]
pipeline = Pipeline(stages=indexer)
transformed = pipeline.fit(nd).transform(nd)
transformed.show()

23/12/01 21:20:05 WARN DAGScheduler: Broadcasting large task binary with size 77.2 MiB


+----------+------+--------------+---------------+----------------+
| productID|rating|    reviewerID|productID_index|reviewerID_index|
+----------+------+--------------+---------------+----------------+
|0001713353|   5.0|A1REUF3A1YCPHM|        83802.0|        102888.0|
|0001713353|   5.0| AVP0HXC9FG790|        83802.0|         66476.0|
|0001713353|   5.0|A324TTUBKTN73A|        83802.0|        422534.0|
|0001713353|   5.0|A2RE7WG349NV5D|        83802.0|         46199.0|
|0001713353|   5.0|A32B7QIUDQCD0E|        83802.0|        749842.0|
|0001713353|   5.0|A3H9YD6K9TVKDP|        83802.0|          1808.0|
|0001713353|   5.0| A3M314LI9OYME|        83802.0|        781735.0|
|0001713353|   5.0|A1YDQQJDRHM0FJ|        83802.0|       1191649.0|
|0001713353|   5.0| AHXMDCU0N15TN|        83802.0|         24901.0|
|0001713353|   5.0|A2M46WTE5TR5WN|        83802.0|        723736.0|
|0001713353|   5.0| ATHTCOG6BB6WK|        83802.0|         11341.0|
|0001713353|   5.0|A3MA68F62DFNVU|        83802.

In [7]:
(training,test)=transformed.randomSplit([0.8, 0.2])

In [8]:
training = training.limit(200)

In [9]:
training.count()

23/12/01 21:20:12 WARN DAGScheduler: Broadcasting large task binary with size 77.2 MiB


200

In [10]:
training.show(20)

23/12/01 21:20:28 WARN DAGScheduler: Broadcasting large task binary with size 77.2 MiB


+----------+------+--------------+---------------+----------------+
| productID|rating|    reviewerID|productID_index|reviewerID_index|
+----------+------+--------------+---------------+----------------+
|0000013765|   5.0|A1NGDVXI2BAG7C|       157227.0|        249355.0|
|0000013765|   5.0|A2SUAM1J3GNN3B|       157227.0|        978506.0|
|0000230022|   5.0|A1CESEWTLAUPMN|       116646.0|        874601.0|
|0000230022|   5.0|A22JN1BC3UQT8J|       116646.0|        692072.0|
|0000230022|   5.0|A24EVFNLA6O0K1|       116646.0|        930177.0|
|0000230022|   5.0|A38AAPXSJN4C5G|       116646.0|           316.0|
|0000230022|   5.0| AFQUVVVDRNXEJ|       116646.0|         65799.0|
|0000230022|   5.0|  AYQGXWZP7S2G|       116646.0|       1660543.0|
|0000913154|   5.0|A28X5I7TL8BAOH|       146667.0|        523018.0|
|0000913154|   5.0|A3S1YQA5N7DWGD|       146667.0|        193171.0|
|0000913154|   5.0|A3S1YQA5N7DWGD|       146667.0|        193171.0|
|0000913154|   5.0| AP25VUIFDC582|       146667.

In [11]:
als=ALS(maxIter=5,regParam=0.09,rank=25,userCol="reviewerID_index",itemCol="productID_index",ratingCol="rating",coldStartStrategy="drop",nonnegative=True)
model=als.fit(training)

23/12/01 21:20:37 WARN DAGScheduler: Broadcasting large task binary with size 77.2 MiB
23/12/01 21:20:52 WARN DAGScheduler: Broadcasting large task binary with size 77.2 MiB
23/12/01 21:21:07 WARN DAGScheduler: Broadcasting large task binary with size 77.2 MiB
23/12/01 21:21:15 WARN DAGScheduler: Broadcasting large task binary with size 77.2 MiB
23/12/01 21:21:20 WARN DAGScheduler: Broadcasting large task binary with size 77.2 MiB
23/12/01 21:21:25 WARN DAGScheduler: Broadcasting large task binary with size 77.2 MiB
23/12/01 21:21:33 WARN DAGScheduler: Broadcasting large task binary with size 77.2 MiB
23/12/01 21:21:38 WARN DAGScheduler: Broadcasting large task binary with size 77.2 MiB
23/12/01 21:21:47 WARN DAGScheduler: Broadcasting large task binary with size 77.2 MiB
23/12/01 21:21:49 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/12/01 21:21:52 WARN DAGScheduler: Broadcasting large task binary with size 77.2 MiB
23/12/01 21:21:57 WARN 

Py4JError: An error occurred while calling o143.fit

In [ ]:
evaluator=RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")
predictions=model.transform(test)
rmse=evaluator.evaluate(predictions)
print("RMSE="+str(rmse))
predictions.show()

In [ ]:
user_recs=model.recommendForAllUsers(20).show(10)

In [ ]:
# import pandas as pd
# recs=model.recommendForAllUsers(10).toPandas()
# nrecs=recs.recommendations.apply(pd.Series) \
#             .merge(recs, right_index = True, left_index = True) \
#             .drop(["recommendations"], axis = 1) \
#             .melt(id_vars = ['reviewerID_index'], value_name = "recommendation") \
#             .drop("variable", axis = 1) \
#             .dropna() 
# nrecs=nrecs.sort_values('reviewerID_index')
# nrecs=pd.concat([nrecs['recommendation'].apply(pd.Series), nrecs['reviewerID_index']], axis = 1)
# nrecs.columns = [
        
#         'ProductID_index',
#         'Rating',
#         'UserID_index'
       
#      ]
# md=transformed.select(transformed['reviewerID'],transformed['reviewerID_index'],transformed['asin'],transformed['asin_index'])
# md=md.toPandas()
# dict1 =dict(zip(md['reviewerID_index'],md['reviewerID']))
# dict2=dict(zip(md['asin_index'],md['asin']))
# nrecs['reviewerID']=nrecs['UserID_index'].map(dict1)
# nrecs['asin']=nrecs['ProductID_index'].map(dict2)
# nrecs=nrecs.sort_values('reviewerID')
# nrecs.reset_index(drop=True, inplace=True)
# new=nrecs[['reviewerID','asin','Rating']]
# new['recommendations'] = list(zip(new.asin, new.Rating))
# res=new[['reviewerID','recommendations']]  
# res_new=res['recommendations'].groupby([res.reviewerID]).apply(list).reset_index()
# print(res_new)